In [ ]:
import os
import json
import re
from itertools import chain, groupby
import pandas as pd
import numpy as np
import random

In [82]:
with open("dataset/T2IS/data_summary.jsonl", "r") as f:
    data = [json.loads(line) for line in f]

# Group by uid then (height, width)
for d in data:
    d['size'] = (d['height'], d['width'])

In [87]:
data = [d for d in data if sum(d['size']) < 512 * 5]
# with open("dataset/T2IS/data_summary_less_than_5.jsonl", "w") as f:
#     for d in subdata:
#         f.write(json.dumps(d) + "\n")

In [99]:
random.shuffle(data)

In [100]:
train_set = pd.DataFrame(data[:int(len(data)*0.5)])
test_set = pd.DataFrame(data[int(len(data)*0.5):])

grouped_train = train_set.groupby(['uid', 'size'])
grouped_test = test_set.groupby(['uid', 'size'])

grouped_train = {
    name: group.to_dict('records')
    for name, group in grouped_train
}
grouped_test = {
    name: group.to_dict('records')
    for name, group in grouped_test
}

In [101]:
len(grouped_train), len(grouped_test)

(46, 46)

In [102]:
[np.unique([len(x) for x in grouped.values()]) for grouped in [grouped_train, grouped_test]]

[array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 11, 12]),
 array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 11, 12])]

In [103]:
output_dir = "dataset/T2IS/train_half_less_than_5"
os.makedirs(output_dir, exist_ok=True)
with open(os.path.join(output_dir, "train_metadata.jsonl"), "w") as f:
    for group in grouped_train.values():
        for item in group:
            f.write(json.dumps(item) + "\n")

with open(os.path.join(output_dir, "test_metadata.jsonl"), "w") as f:
    for group in grouped_test.values():
        for item in group:
            f.write(json.dumps(item) + "\n")